<a href="https://colab.research.google.com/github/Sambarlasagna/Deep_reinf_learning/blob/main/Lunar_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [16]:
import gymnasium
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

Action 0: Do nothing,

Action 1: Fire left orientation engine,

Action 2: Fire the main engine,

Action 3: Fire right orientation engine.


In [17]:
!pip install Box2d

In [18]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v3")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 3
Action taken: 0
Action taken: 2
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 2
Action taken: 0
Action taken: 3
Action taken: 1
Action taken: 0
Action taken: 2
Action taken: 2
Action taken: 3
Action taken: 2
Action taken: 1


In [19]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v3")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-1.5837035  -0.11319739  4.1470027   6.8737683   1.9246432  -1.4723064
  0.75386935  0.42182374]


In [20]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [21]:
# Vectorized + monitored environment
env = make_vec_env(
    lambda: Monitor(gym.make("LunarLander-v3")),
    n_envs=16
)

In [22]:
# Create model
model = PPO(
    "MlpPolicy",
    env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1
)

Using cuda device


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [23]:
model.learn(total_timesteps=1_000_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.2     |
|    ep_rew_mean     | -167     |
| time/              |          |
|    fps             | 2817     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 85.1        |
|    ep_rew_mean          | -135        |
| time/                   |             |
|    fps                  | 2058        |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.011069989 |
|    clip_fraction        | 0.0727      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.0025      |
|    learning_rate        | 0.

In [24]:
model.save("ppo-LunarLander-v3")

In [25]:
model = PPO.load("ppo-LunarLander-v3", env=env)

In [26]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

In [27]:
eval_env = Monitor(gym.make("LunarLander-v3"))
obs, _ = eval_env.reset()

for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    eval_env.render()

    if terminated or truncated:
        obs, _ = eval_env.reset()

/usr/local/lib/python3.12/dist-packages/gymnasium/envs/box2d/lunar_lander.py:672: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("LunarLander-v3", render_mode="rgb_array")
  gym.logger.warn(


In [28]:
mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True
)

print(f"Mean reward: {mean_reward}")
print(f"Std reward: {std_reward}")

Mean reward: 250.47552299999998
Std reward: 19.835419079985112


In [29]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [30]:
!pip install pyvirtualdisplay huggingface-sb3 stable-baselines3 gymnasium

In [33]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from huggingface_sb3 import package_to_hub

env_id = "LunarLander-v3"
model_name = "PPO_LunarLander_v3"
model_architecture = "PPO"

repo_id = "Sambarlasagna/PPO_LunarLander_v3"
commit_message = "Upload PPO LunarLander-v3 trained agent"

# Evaluation env (for video & metrics)
eval_env = DummyVecEnv([
    lambda: gym.make(env_id, render_mode="rgb_array")
])

package_to_hub(
    model=model,
    model_name=model_name,
    model_architecture=model_architecture,
    env_id=env_id,
    eval_env=eval_env,
    repo_id=repo_id,
    commit_message=commit_message,
)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpe9_67quu/-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /tmp/tmpe9_67quu/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpe9_67quu/-step-0-to-step-1000.mp4



...

  ...r_v3/policy.optimizer.pth: 100%|##########| 88.9kB / 88.9kB            

  ...LunarLander_v3/policy.pth: 100%|##########| 44.1kB / 44.1kB            

  ...ch/PPO_LunarLander_v3.zip: 100%|##########|  149kB /  149kB            

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Sambarlasagna/PPO_LunarLander_v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/Sambarlasagna/PPO_LunarLander_v3/commit/e1aa282a11d6d2f06721b3dbe494ddcc4b6b822f', commit_message='Upload PPO LunarLander-v3 trained agent', commit_description='', oid='e1aa282a11d6d2f06721b3dbe494ddcc4b6b822f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sambarlasagna/PPO_LunarLander_v3', endpoint='https://huggingface.co', repo_type='model', repo_id='Sambarlasagna/PPO_LunarLander_v3'), pr_revision=None, pr_num=None)